# 4/ End to End SDP Production Pipeline for product review and classification with SQL AI QUERY


In this demo, we will create a production ready pipeline for the product review and classification using Spark Declarative Pipelines framework

- `reviews_annotated_silver` table: Join review and customer data, annotate reviews
- `review_answer_gold` table: generate response per customer review

Please note, we need to use the full Unity Catalog path of the AI SQL Functions


<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/product/sql-ai-functions/sql-ai-query-function-flow.png" width="1000">


<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=DBSQL&org_id=984752964297111&notebook=%2F04-create-end-to-end-SDP-workflow&demo_name=sql-ai-functions&event=VIEW&path=%2F_dbdemos%2FDBSQL%2Fsql-ai-functions%2F04-create-end-to-end-SDP-workflow&version=1&user_hash=d0474b04fa5f647b58e56efa05dd4a6e679929fda0191106826e4a09d21dcede">

# Create `reviews_annotated_silver` streaming table

In [0]:
CREATE OR REFRESH STREAMING TABLE reviews_annotated_silver(
  CONSTRAINT valid_customer_id EXPECT (customer_id IS NOT NULL)
) 
COMMENT "Annotate review with Llama 3.3 70B"
AS SELECT
  * EXCEPT (review_annotated), review_annotated.* FROM (
      SELECT *, dbacademy.himanshu_gupta.ANNOTATE_REVIEW(review) AS review_annotated
        FROM stream(fake_reviews))
    INNER JOIN stream(fake_customers) using (customer_id)

# Create `review_answer_gold` materialized view

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW reviews_answer_gold 
COMMENT "Create review answer and response"
AS SELECT 
  *,
  dbacademy.himanshu_gupta.generate_response(firstname, lastname, order_count, product_name, followup_reason) AS response_draft
FROM live.reviews_annotated_silver where followup='Y'